In [18]:
from EvalWhisper import *
import soundfile as sf

In [19]:
import argparse
import json
import pathlib
from pathlib import Path
import statistics

from EvalWhisper import EvalWhisper

from datasets import load_dataset
from tqdm import tqdm

In [20]:
model_size = "large"
model_checkpoint = f"openai/whisper-{model_size}"
processor_checkpoint = [f"openai/whisper-{model_size}"]
#load processor and model
print(f"Loading model . . . ({model_checkpoint})")
whisper_evaluator = EvalWhisper(model_checkpoint, *processor_checkpoint)
print(f"Loaded model")


Loading model . . . (openai/whisper-large)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model


In [21]:
ds = load_dataset("librispeech_asr", split="validation.clean", streaming=True)
print(f"Loaded data")

Loaded data


In [22]:
list(ds.take(1))[0]

{'file': '2277-149896-0000.flac',
 'audio': {'path': '2277-149896-0000.flac',
  'array': array([ 0.00186157,  0.0005188 ,  0.00024414, ..., -0.00097656,
         -0.00109863, -0.00146484]),
  'sampling_rate': 16000},
 'text': "HE WAS IN A FEVERED STATE OF MIND OWING TO THE BLIGHT HIS WIFE'S ACTION THREATENED TO CAST UPON HIS ENTIRE FUTURE",
 'speaker_id': 2277,
 'chapter_id': 149896,
 'id': '2277-149896-0000'}

In [6]:
! 

In [15]:
def get_audio(output_path, whisper_evaluator, inputs, mask_ratios, mode_value, what_to_mask_list, sampling_rate):
    output_audio_list = []
    
    
    for idx, input in tqdm(enumerate(inputs)):
        masked_spectrograms = []
        original_spectrogram = whisper_evaluator.get_spectrogram(input)
        # print(f"os:{original_spectrogram.shape}")
        sf.write(f"{output_path}/{idx}_original_all.wav", input["audio"]["array"], sampling_rate) #whisper_evaluator.top_r_features(input, r=1.0, mode="retain", where="top").detach().numpy()
        sf.write(f"{output_path}/{idx}_{1.0}_all.wav", whisper_evaluator.sonify(original_spectrogram), sampling_rate) #whisper_evaluator.top_r_features(input, r=1.0, mode="retain", where="top").detach().numpy()
        with open(f"{output_path}/{idx}_original_transcription.txt", "w") as op_file:
            op_file.write(input["text"] + "\n")
        break
        for mask_ratio in tqdm(mask_ratios):
            mask_list = []
            for mask in what_to_mask_list:
                masked_spectrogram = whisper_evaluator.top_r_features(input, r=mask_ratio, mode=mode_value, where=mask)
                masked_audio = whisper_evaluator.sonify(masked_spectrogram.detach().numpy())
                mask_list.append(masked_audio)
                sf.write(f"{output_path}/{idx}_{mask_ratio}_{mask}.wav", masked_audio, sampling_rate)
            masked_spectrograms.append(mask_list)

    

In [17]:
output_path = Path("./output_audio_new")
pathlib.Path.mkdir(output_path, exist_ok=True)
mask_ratios = [0.8, 0.6, 0.4, 0.2]
mode_value = "retain"
what_to_mask_list = ["top", "bottom", "random"]
num_samples = 3
skip_to_index = 510
inputs = []
for sample in tqdm(ds.skip(skip_to_index).take(num_samples)):
    inputs.append(sample)
get_audio(whisper_evaluator, inputs, mask_ratios, mode_value, what_to_mask_list)

0it [00:28, ?it/s]


KeyboardInterrupt: 